In [1]:
import pandas as pd

# Initialize empty lists to store data
def txt_to_pd(input_txt): 
    run = []
    process = []
    avg_latency = []

    # Open the text file
    with open(input_txt, 'r') as file:
        lines = file.readlines()

        # Initialize variables to hold current run and process numbers
        current_run = None
        current_process = None

        # Iterate through each line in the file
        for line in lines:
            if line.startswith("Run number:"):
                current_run = int(line.split(":")[1].strip())
                
            elif line.startswith("Processes:"):
                current_process = int(line.split(":")[1].strip())
                
            
            elif line[0] == ' ':
                number = float(line.strip())
                avg_latency.append(number)
                run.append(current_run)
                process.append(current_process)

    data = {
    'Run': run,
    'Process': process,
    'Avg Latency(us)': avg_latency}

    data = pd.DataFrame(data)

    return data 

df = txt_to_pd('barrier.txt')
bar_df = df.drop(['Run'], axis=1)
bar_df

,Process,Avg Latency(us)
0,2,0.29
1,4,0.45
2,6,1.24
3,8,1.18
4,10,1.48
...,...,...
251,248,120.95
252,250,128.82
253,252,119.00
254,254,96.43


In [2]:
processes = [i for i in range(2, 257, 2)]

def summary_stats(df, processes):
    avg = []
    std = []
    proc = []
    for process in processes: 
        subdf = df[(df['Process'] == process)]
        avg.append(subdf['Avg Latency(us)'].mean())
        std.append(subdf['Avg Latency(us)'].std())
        proc.append(process)

    d = {'processes': proc,
    'avg time': avg,
    'standard deviation': std}

    data = pd.DataFrame(d)

    return data

barrier = summary_stats(bar_df, processes)
barrier

,processes,avg time,standard deviation
0,2,0.295,0.007071
1,4,0.450,0.000000
2,6,1.145,0.134350
3,8,1.215,0.049497
4,10,1.405,0.106066
...,...,...,...
123,248,136.310,21.722320
124,250,112.650,22.867833
125,252,121.280,3.224407
126,254,98.895,3.486036


In [3]:
# CPUs latencies
ccx = 0.14 # 3
ccd = 0.31 # 7 
numa_ccd = 0.34 # 15 
socket = 0.44 # 63
node = 0.66 # 127
d_node = 1.82 # 255

ccx_l = [ccx] * 2
ccd_l = [ccd] * 2
numa_ccd_l = [numa_ccd] * 4
socket_l = [socket] * 24
node_l = [node] * 32
d_node_l = [d_node] * 64

latency_list = ccx_l + ccd_l + numa_ccd_l + socket_l + node_l + d_node_l

barrier['cpu latency'] = latency_list
barrier['effective time'] = barrier['avg time'] - barrier['cpu latency']
barrier

,processes,avg time,standard deviation,cpu latency,effective time
0,2,0.295,0.007071,0.14,0.155
1,4,0.450,0.000000,0.14,0.310
2,6,1.145,0.134350,0.31,0.835
3,8,1.215,0.049497,0.31,0.905
4,10,1.405,0.106066,0.34,1.065
...,...,...,...,...,...
123,248,136.310,21.722320,1.82,134.490
124,250,112.650,22.867833,1.82,110.830
125,252,121.280,3.224407,1.82,119.460
126,254,98.895,3.486036,1.82,97.075


In [4]:
import plotly.express as px
import plotly.graph_objects as go

# Plot the execution time 

fig = px.line(barrier, x='processes', y='avg time', title="Broadcast: Binary Tree")
fig

In [5]:
# Compute the slopes using linear regression
overhead = ((barrier['effective time'].iloc[-1] - barrier['effective time'].iloc[0]) / 
                  (barrier['processes'].iloc[-1] - barrier['processes'].iloc[0]))

barrier['overhead'] = overhead
barrier


,processes,avg time,standard deviation,cpu latency,effective time,overhead
0,2,0.295,0.007071,0.14,0.155,0.408169
1,4,0.450,0.000000,0.14,0.310,0.408169
2,6,1.145,0.134350,0.31,0.835,0.408169
3,8,1.215,0.049497,0.31,0.905,0.408169
4,10,1.405,0.106066,0.34,1.065,0.408169
...,...,...,...,...,...,...
123,248,136.310,21.722320,1.82,134.490,0.408169
124,250,112.650,22.867833,1.82,110.830,0.408169
125,252,121.280,3.224407,1.82,119.460,0.408169
126,254,98.895,3.486036,1.82,97.075,0.408169


In [12]:
barrier['hockney'] = (barrier['processes'] - 1) * barrier['cpu latency']
fig = px.scatter(barrier, x='processes', y='avg time', title="Barrier")
t = go.Line(x=barrier["processes"], y=barrier['hockney'], name="hockey")
fig.add_trace(t)
fig.add_vline(x=64, line_dash="dash", line_color="red", annotation_text="Process 64")
fig.add_vline(x=128, line_dash="dash", line_color="blue", annotation_text="Process 128")


/home/ziyangfu/Code/rec_system/rec/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [18]:
barrier['logP'] = (barrier['processes'] - 2) + (1 * ((barrier['cpu latency'] * 2) + (barrier['overhead'] * 2)))
pt2pt = 0.155

In [19]:
fig = px.scatter(barrier, x='processes', y='avg time', title="Barrier")
t = go.Line(x=barrier["processes"], y=barrier['logP'], name="logP")
fig.add_trace(t)

fig.add_vline(x=64, line_dash="dash", line_color="red", annotation_text="Process 64")
fig.add_vline(x=128, line_dash="dash", line_color="blue", annotation_text="Process 128")


/home/ziyangfu/Code/rec_system/rec/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


